In [ ]:
# dataframe
import pandas as pd

df = pd.read_csv('./payload/train.csv')

In [10]:
paths = [
    'history/runs/run_vit_base_20241207_113554_ep28/',
    'history/runs/run_efficientnet_20241205_114438_ep7/',
    'history/runs/run_resnet50_20241208_231720_ep6/',
    'history/runs/run_resnet101_20241204_201138_ep14/',
    'history/runs/run_vit_mediumd_20241209_002034_ep26/',
]
# example path history\runs\run_efficientnet_20241205_114438_ep7\models\best_auc_model.pth

import os
 
# pull the best_auc_model.pth from each path and save it to ./models and rename it accordingly to the modelname COPY!!!
for path in paths:
    model_name = path.split('/')[2].split('_')[1] + '.pth'
    # if there's no best_auc_model.pth in the path, use best_loss_model.pth instead
     
    if os.path.exists(path + '/models/best_auc_model.pth'):
        os.system(f'copy {path}/models/best_auc_model.pth ./models/{model_name}')
    else:
        os.system(f'copy {path}/models/best_loss_model.pth ./models/{model_name}')

    print(f'Copied {model_name} to ./models')


Copied vit.pth to ./models
Copied efficientnet.pth to ./models
Copied resnet50.pth to ./models
Copied resnet101.pth to ./models
Copied vit.pth to ./models
